In [1]:
sc

In [2]:
dfhtag1 = sqlContext.read.load('stream_twt_may11.jsonl', format='json')
dfhtag1.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |

In [3]:
#Retweeted Years
import pyspark.sql.functions as sf
timeline = dfhtag1.select('quoted_status.created_at').dropna()
timeline = timeline.select(timeline.created_at.substr(sf.length(timeline.created_at)-25, sf.length(timeline.created_at)-27).alias("Month")\
           ,timeline.created_at.substr(sf.length(timeline.created_at)-4, sf.length(timeline.created_at)).alias("Year"))
timeline = timeline.filter(timeline.Year <= 2013)
timeline = timeline.select(sf.concat(timeline.Month, timeline.Year).alias('date')) 
#timeline.show(truncate =False, n=100)

In [4]:
import pyspark.sql.functions as sf
timeline = timeline.groupBy('date').count().orderBy('count', ascending = False)
#timeline.show(truncate=False, n=1000)

In [5]:
import pyspark.sql.functions as sf
timeline2 = dfhtag1.select('quoted_status.text','quoted_status.created_at').dropna()
timeline2 = timeline2.select('text',timeline2.created_at.substr(sf.length(timeline2.created_at)-25, sf.length(timeline2.created_at)-27).alias("Month")\
           ,timeline2.created_at.substr(sf.length(timeline2.created_at)-4, sf.length(timeline2.created_at)).alias("Year"))
timeline2 = timeline2.filter(timeline2.Year <= 2013)
timeline2 = timeline2.select('text',sf.concat(timeline2.Month, timeline2.Year).alias('date_'))                 
#timeline2.show(truncate =False, n=100)

In [7]:
result = timeline.join(timeline2, timeline.date == timeline2.date_)
result = result.drop('date_')

result.show(truncate=False, n=100)

+--------+-----+--------------------------------------------------------------------------------------------------------------------------------------------+
|date    |count|text                                                                                                                                        |
+--------+-----+--------------------------------------------------------------------------------------------------------------------------------------------+
|Jul 2011|7    |I'm....I'm a what?!                                                                                                                         |
|Jul 2011|7    |I'm....I'm a what?!                                                                                                                         |
|Jul 2011|7    |I'm....I'm a what?!                                                                                                                         |
|Jul 2011|7    |I'm....I'm a what?!                 

In [8]:
result.toPandas().to_csv('timelines/may11quote_timeline_wtxt.csv', encoding='utf-8', index=False)